In [6]:
import numpy
from pybit.unified_trading import HTTP
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from models.TS.GRUPipeline import GRUNetwork
import time
from PyEMD import CEEMDAN
from torch import optim
from torch import nn
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

In [2]:
SYMBOL = 'SOLUSDT'
API_KEY = 'Z8QsHgyTpwinr9K2Ya'
API_SECRET = '5QfjbglBfFK5A8RqGvcvdw8hCF4Yjv5sByOn'
cl = HTTP(
    
)

In [3]:
START_TIMESTAMP = '1711929617184'
time_series = 1000 #  количество изначальных временных рядов в разные промежутки времени, с количеством измерений t_s_l
time_series_length = 20

time_stamps = [START_TIMESTAMP]

train_data = np.ndarray(shape=(time_series, time_series_length))
volumes = np.ndarray(shape=time_series_length*time_series)
all_price = np.ndarray(shape=time_series_length*time_series)

for i in range(time_series):
    kline = cl.get_kline(
        category="spot",
        symbol=SYMBOL,
        start=time_stamps[-1],
        interval=1,
        limit = time_series_length
    )
    if i % 115 == 0:
        time.sleep(5)
    time_stamps.append(str( int(kline.get('result', []).get('list', [])[0][0]) + 10000))
    
    volumes[i*time_series_length:(i+1)*time_series_length] = np.array([i[5] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    all_price[i*time_series_length:(i+1)*time_series_length] = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    
    prices = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)         
    train_data[i] = np.array(prices, dtype=float)

In [31]:
ceem = CEEMDAN(parallel=True)
imfs = ceem(torch.tensor(train_data[0].reshape(-1, 1)).cpu().numpy().squeeze())  # проводим EMD на временном ряде
imfs = imfs[:3]  # берем только три первых IMFs
imfs = torch.tensor(imfs).transpose(0, 1)
print(imfs)

tensor([[ 4.5886e-02, -7.8835e-02,  2.0220e+02],
        [ 1.0631e-01, -1.2188e-01,  2.0219e+02],
        [-1.1752e-01, -6.1703e-02,  2.0216e+02],
        [ 1.1661e-01,  6.7749e-02,  2.0213e+02],
        [-9.0812e-02,  1.4425e-01,  2.0209e+02],
        [ 7.4085e-02,  1.1219e-01,  2.0204e+02],
        [-5.6436e-02,  1.7565e-02,  2.0200e+02],
        [-8.0327e-02, -9.3408e-02,  2.0195e+02],
        [-1.6028e-01, -1.7977e-01,  2.0191e+02],
        [ 1.8806e-01, -2.0763e-01,  2.0187e+02],
        [-3.4033e-03, -1.8043e-01,  2.0183e+02],
        [-7.8074e-02, -1.1550e-01,  2.0180e+02],
        [-1.9752e-01, -3.1148e-02,  2.0178e+02],
        [ 1.9564e-01,  5.5502e-02,  2.0176e+02],
        [-1.6601e-01,  1.3214e-01,  2.0174e+02],
        [-1.8631e-01,  1.9288e-01,  2.0173e+02],
        [-2.0620e-01,  2.3893e-01,  2.0173e+02],
        [-2.0621e-01,  2.7107e-01,  2.0173e+02],
        [ 7.3134e-02,  2.9011e-01,  2.0173e+02],
        [ 1.7224e-01,  2.9591e-01,  2.0173e+02]], dtype=torch.float64

In [51]:
train_size = int(time_series * 0.8)
prediction_length = 10
idxs = np.random.choice((time_series-1), size=train_size)
x_train, y_train, = torch.tensor(train_data[idxs]).unsqueeze(-1), torch.tensor(train_data[idxs+1][:, :prediction_length]).unsqueeze(-1)
print(x_train.shape, y_train.shape)
print(idxs.shape)
last_idxs = np.array([i for i in range(0, time_series-1) if not np.isin(i, idxs)])
# Получение тестовых данных


x_test, y_test = torch.tensor(train_data[last_idxs]).unsqueeze(-1), torch.tensor(train_data[last_idxs + 1][:, :prediction_length]).unsqueeze(-1)
print(x_test.shape)

torch.Size([800, 20, 1]) torch.Size([800, 10, 1])
(800,)
torch.Size([447, 20, 1])


In [56]:
torch.save(x_train, f'trainX_{SYMBOL}.pt')
torch.save(y_train, f'trainY_{SYMBOL}.pt')
torch.save(x_test, f'testX_{SYMBOL}.pt')
torch.save(y_test, f'testY_{SYMBOL}.pt')

In [52]:
dataset = TensorDataset(x_train, y_train)
batch_size = 5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [54]:
gru = GRUNetwork(1, 30, 2, 10, 1)

criterion = nn.MSELoss()
optimizer = optim.Adam(gru.parameters(), lr=0.01)

# Обучение модели
num_epochs = 1000

for epoch in range(num_epochs):
    for i, (batch_x, batch_y) in enumerate(dataloader):
        gru.train()
        outputs = gru(batch_x)
        optimizer.zero_grad()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.Size([20, 3, 1])
torch.Size([20, 3, 1])
torch.Size([20, 3, 1])
torch.Size([20, 2, 1])
torch.Size([20, 3, 1])


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1